In [1]:
import os, sys
import numpy as np

from pathlib import Path

from astropy.io import fits, ascii
from astropy.table import Table, Column, vstack, hstack, join

from scipy import interpolate
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

sys.path.append("./py")
from utils import *

sys.path.append("/Users/aberti/Desktop/research")
from plotutils import get_corners, fig_labels, get_colors, plot_settings
plt.rcParams.update(**plot_settings)

plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.family"] = "STIXGeneral"

from params import BASEDIR, DATADIR, SIMDIR, H0, Om0
from params import get_boxsize, get_zsnap_data, get_sham_var_bins, get_abs_mag_lim, get_abs_mag_bins_clust

#-- bins for clustering (data and mocks)
from params import H0 #, bin_file_comoving


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:550: UserWarning: matplotlibrc text.usetex can not be used unless ghostscript-9.00 or later is installed on your system
  % gs_req)


<a class="anchor" id="outline"></a>

* [Load mock catalog](#load_mock)
* [Color and magnitude distribution comparison figure](#color-mag-dist-fig)
* [Luminosity function comparison figure](#lum-func-fig)
* [Color assignment](#color)
  * [Add scatter to colors](#color_scatter)
* [LRG flagging](#lrg)


# Set parameters

In [2]:
band   = "Mz"
zrange = (0.4,0.5)

print(band)
print(zrange)

zmag_lim = 20.7
sim_tag  = "mdpl2"
sham_tag = "vpeak"
d        = "south"
pimax    = 150
h        = H0/100.


z_snaps, snap_nums, snaps = get_zsnap_data(sim_tag)

vol_sim  = get_boxsize(sim_tag)**3
a_snaps = [snaps["scale"][snaps["snapnum"]==sn].data[0] for sn in snap_nums]

zmin, zmax  = zrange
cat_tag     = get_cat_tag(d,zrange)
abs_mag_lim = get_abs_mag_lim(zmin, band)
zsim        = np.array(z_snaps)[ np.round(z_snaps,1)==zmin ][0]
zsnap_tag   = get_zsnap_tag(zsim)
zmag_tag    = get_zmag_tag(zmag_lim)

MW1lim = get_abs_mag_lim(zmin,"MW1")
Mrlim  = get_abs_mag_lim(zmin,"Mr")
Mzlim  = get_abs_mag_lim(zmin,"Mz")
if band=="MW1":
    abs_mag_lim_tag = get_MW1_tag(MW1lim)
    abs_mag_range   = np.array((-24.5,-21.99))
    model_mag_range = np.array((-22.25,-25.35))
elif band=="Mr":
    abs_mag_lim_tag = get_Mr_tag(Mrlim)
    abs_mag_range   = np.array((-23,-20.49))
    model_mag_range = np.array((-20.8,-23.85))
elif band=="Mz":
    abs_mag_lim_tag = get_Mz_tag(Mzlim)
    abs_mag_range   = np.array((-23.75,-21.24))
    model_mag_range = np.array((-21.6,-24.6))
else:
    raise Exception()
    
abs_mag_bins_clust = get_abs_mag_bins_clust(zmin,band)[::-1]#[:-1]
    
# print(abs_mag_bins_clust)


Mz
(0.4, 0.5)


In [ ]:
def get_cs(band,zmin):
    if band=="Mz": return 0.6
    elif band=="MW1":
        if zmin==0.6: return 0.6
        elif (zmin==0.4) | (zmin==0.5): return 0.8
        else: raise Exception
    else: raise Exception

cs = get_cs(band,zmin)
cs_tag = f"cs{cs}".replace(".","p")

fname = f"{BASEDIR}/data/hod_zsim{str(zsim).replace('.','p')}_{band}.npy"
# fname = f"{BASEDIR}/data/hod_zsim{str(zsim).replace('.','p')}_{band}_{cs_tag}.npy"
print(fname)

this_hod_data = Table(np.load(fname))


In [ ]:
show = Table(data=[this_hod_data["mvir_mids"],
                  np.round(this_hod_data["mean_num_cen_opt_lrg_per_parent_mvir_bin"],2),
                  np.round(this_hod_data["mean_num_cen_IR_lrg_per_parent_mvir_bin"],2)],
             names=("Mvir", "Ncen_LRG-opt", "Ncen_LRG-IR"))

print(zmin, band, (cs if "cs" in fname else ""))

show[show["Mvir"] > 13.5]


<a class="anchor" id="load_mock"></a>
# Load mock

[BACK TO TOP](#outline)

[JUMP TO COLOR SCATTER](#color_scatter)


In [ ]:
%%time

# f = f"{BASEDIR}/mocks/{sim_tag}/{sham_tag}/{d}/{zsnap_tag}_{abs_mag_lim_tag}"
f = f"{BASEDIR}/mocks/{sim_tag}/{sham_tag}/{d}/color_scatter/{zsnap_tag}_{abs_mag_lim_tag}"

# galcat_fname = f"{f}_galcat-hist-LRG.npy"
galcat_fname = f"{f}_galcat-LRG.npy"

print(galcat_fname)

cat = Table(np.load(galcat_fname))


In [ ]:
if band=="Mz":
    if zmin==0.6:
        abs_mag_bins = np.concatenate( [np.round(np.arange(abs_mag_lim,-24.8,-0.05),2),
                                       [-24.8,-24.9,-25,-25.5,np.nanmin(cat[band])] ])
    elif (zmin==0.5) | (zmin==0.4):
        abs_mag_bins = np.concatenate( [np.round(np.arange(abs_mag_lim,-24.8,-0.05),2),
                                       [-24.8,-24.9,-25,np.nanmin(cat[band])] ])
elif band=="MW1":
    if zmin==0.6:
        abs_mag_bins = np.concatenate( [np.round(np.arange(abs_mag_lim,-26.1,-0.05),2),
                                       [-26.1,np.nanmin(cat[band])] ])
    elif zmin==0.5:
        abs_mag_bins = np.concatenate( [np.round(np.arange(abs_mag_lim,-25.8,-0.05),2),
                                       [-25.8,-26,np.nanmin(cat[band])] ])
    elif zmin==0.4:
        abs_mag_bins = np.concatenate( [np.round(np.arange(abs_mag_lim,-25.5,-0.05),2),
                                       [-25.5,-25.6,-25.7,np.nanmin(cat[band])] ])


In [ ]:
#-- data for color distributions

t = Table(np.load(f"{BASEDIR}/mags_{cat_tag}.npy"))


In [ ]:
#-- color assigment to improve

if band=="Mz":
    color_bins = np.arange(0,3.01,0.01)
elif band=="MW1":
    color_bins = np.arange(0,4.51,0.01)

color_cens = 0.5*(color_bins[1:] + color_bins[:-1])

# fig, ax = plt.subplots(1, 1, figsize=(8,8))

colors_data = (t["r"] - t["Kr"]) - (t[band[1:]] - t[f"K{band[1:]}"])
abs_mag_mask_data = t[band] <= abs_mag_lim

# ax.hist(cat[f"r-{band[1:]}"][cat[band] <= abs_mag_lim], bins=color_bins, density=True, color="gray", alpha=0.5, label="model")
# ax.hist( colors_data[abs_mag_mask_data], bins=color_bins, density=True, histtype="step", color="k", lw=2, label="target")
# ax.legend()

# ax.set_xlabel(f"$r-{band[1:]}$")
# ax.set_xlim(np.min(color_bins),np.max(color_bins))

# plt.tight_layout()
# plt.show()


<a class="anchor" id="color"></a>
# (improved) color assignment

[BACK TO TOP](#outline)


In [ ]:
# mask = snaps["redshift"] < 8

# ds = snaps["redshift"][mask][:-1]-snaps["redshift"][mask][1:]

# print(np.min(ds),np.max(ds))


In [ ]:
# #-- smooth zstarve values to eliminate effect discreteness from simulation timesteps

# bins = np.arange(0,16,0.05)

# fig, ax = plt.subplots(1, 1, figsize=(8,8))

# zstarve = cat["zstarve"].data

# # print(len(zstarve) - len(zstarve[ (zstarve >= np.nanmin(zstarve)) & (zstarve <= np.nanmax(zstarve)) ]))

# #-- sigma for zstarve smoothing is free parameter
# #-- 0.05 clipped at 3-sigma produces smoothed distribution using same bins as unsmoothed
# # zstarve_smoothed = np.random.normal(zstarve, 0.05)
# zstarve_smoothed = stats.truncnorm.rvs(-3, 3, zstarve, 0.05)
# # zstarve_smoothed[zstarve_smoothed < np.nanmin(zstarve)] = np.nanmin(zstarve) #-- don't do this
# zstarve_smoothed[zstarve_smoothed > np.nanmax(zstarve)] = np.nanmax(zstarve)

# if "zstarve_smoothed" in cat.colnames:
#     cat.remove_column("zstarve_smoothed")
# cat.add_column(Column(zstarve_smoothed, name="zstarve_smoothed"))

# ax.hist(zstarve, bins=bins, density=True, color="gray", alpha=0.5, label="unsmoothed")
# ax.hist(zstarve_smoothed, bins=bins, density=True, color="k", histtype="step", lw=2, label="smoothed")

# ax.set_xlim(0,10)
# ax.set_xlabel(r"$z_{\rm starve}$", fontsize=24)

# ax.legend()

# plt.tight_layout()
# plt.show()


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8,8))

# ax.scatter(cat["zstarve"][::10], cat["zstarve_smoothed"][::10], alpha=0.5, s=2)
# ax.set_xlim(1,2)
# ax.set_ylim(1,2)

# plt.tight_layout()
# plt.show()


In [ ]:
zstarve_smoothed = cat["zstarve_smoothed"].data

zstarve_min = np.floor(100*np.nanmin(zstarve_smoothed))/100
zstarve_max =  np.ceil(100*np.nanmax(zstarve_smoothed))/100

mask = (zstarve_smoothed > zstarve_min) & (zstarve_smoothed < zstarve_max)

# zstarve_bins = np.quantile(zstarve_smoothed[mask], np.arange(0,1.01,0.01))
zstarve_bins = np.arange(0,16,0.01)
zstarve_cens = 0.5*(zstarve_bins[1:] + zstarve_bins[:-1])

# zstarve_cens


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8,8))

# ax.hist(cat[band], bins=abs_mag_bins[::-1], color="gray", alpha=0.5)

# ax.set_xlim(-21,-26.5)
# ax.set_ylim(0,5000)
# plt.tight_layout()
# plt.show()


In [ ]:
color_cname = f"r-{band[1:]}"
# if color_cname in cat.colnames:
#     cat.remove_column(color_cname)
# cat.add_column(Column(np.zeros(len(cat)), name=color_cname))

abs_mag_assigned = cat[f"{band}"]
# zstarve_smoothed = cat["zstarve_smoothed"]

data_colors  = (t["r"].data-t["Kr"].data) - (t[band[1:]].data-t[f"K{band[1:]}"].data)
abs_mag_data = t[band].data


In [ ]:
from scipy.signal import savgol_filter

fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.set_xlim(0.5,1.2)
ax.set_xticks(np.arange(0.5,1.21,0.1))

ax1 = ax.twiny()
ax1.semilogx()
ax1.set_xlim(0.6,10)
ax1.set_xticks([0.6,1.0,2.0,3.0,4.0,6.0,10.0])
ax1.set_xticklabels([f"{i:.1f}" for i in ax1.get_xticks()])
ax1.set_ylim(-0.025,1.15)

c_opt,_ = get_colors(2)

for i in range(len(abs_mag_bins)-1):
    abs_mag_min, abs_mag_max = abs_mag_bins[i:i+2]
    abs_mag_bin_label = f"{abs_mag_min:.2f}_{abs_mag_max:.2f}".replace('.','p').replace('-','n')
    
    abs_mag_mask_model = (abs_mag_assigned < abs_mag_min) & (abs_mag_assigned >= abs_mag_max)
    
    this_bin_zstarve = zstarve_smoothed[abs_mag_mask_model]
    H,_ = np.histogram(this_bin_zstarve, bins=zstarve_bins)
    this_bin_zstarve_cdf = np.cumsum(H)/np.cumsum(H)[-1]

    this_bin_abs_mag_mask_data = (abs_mag_data < abs_mag_min) & (abs_mag_data >= abs_mag_max)
    
    this_bin_color = data_colors[this_bin_abs_mag_mask_data]
    H,_ = np.histogram(this_bin_color, bins=color_bins)
    this_bin_color_cdf = np.cumsum(H)/np.cumsum(H)[-1]

    if ((i+1)%2==0) & (i < 58):
        ax1.plot(zstarve_cens, savgol_filter(this_bin_zstarve_cdf,7,3), label="", color="gray", alpha=0.1)
        ax.plot(color_cens, this_bin_color_cdf, label="", color=c_opt, alpha=0.2, ls=":")
    
    if i==28:
        label = "$^{0.43}(r-z)\ \ (-23.0 > ^{0.43}M_z > -23.05)$"
        ax1.plot((0,0), (1,1), label=label, color=c_opt, lw=2, ls=":")
        ax.plot(color_cens, this_bin_color_cdf, color=c_opt, lw=2.5, ls=":")
        ax1.plot(zstarve_cens, savgol_filter(this_bin_zstarve_cdf,7,3), label=r"halo $z_{\rm starve}$", color="gray", lw=1.5)
        
ax.set_xlabel("model $^{0.43}(r-z)$ color", fontsize=20)
ax1.set_xlabel("redshift", fontsize=20)
ax.set_ylabel("cumulative distribution", fontsize=20)

ax1.legend(loc=2, handletextpad=0.3, fontsize=18, handlelength=1.75, labelspacing=0)

# x0, y0 = 2.585, ax.get_ylim()[-1]
# x1, y1 = 2.585, 0.50
x0, y0 = 0.863, ax.get_ylim()[-1]
x1, y1 = 0.863, 0.50
x2, y2 = 0.96, 0.50
x3, y3 = 0.96, ax.get_ylim()[0]

kwargs = dict(length_includes_head=True, width=0.005, head_width=0.025, alpha=0.3, color="magenta")
ax.arrow(x0,y0,0,y1-y0,**kwargs)
ax.arrow(x2,y2,0,y3-y2,**kwargs)
kwargs["width"] = 0.01
kwargs["head_width"] = 0.045
kwargs["head_length"] = 0.025
ax.arrow(x1,y1,x2-x1,0,**kwargs)

ax.plot(0.76*np.ones(2),(1.015,1.095),color="k",lw=0.75)

for ax in (ax, ax1):
    ax.tick_params(axis="both", which="major", labelsize=18)

fname = f"{BASEDIR}/figures/model_color_CDF_{cat_tag[:-6]}_{abs_mag_lim_tag}_zstarve.pdf"
print(fname)

# plt.savefig(fname, bbox_inches="tight", pad_inches=0.1)#, dpi=300)

plt.tight_layout()
plt.show()


In [ ]:
cs = get_cs(band, zmin)
cs_tag = f"cs{cs}".replace(".","p")

fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.set_xlim(-0.4,0.8)
ax.set_ylim(0.2,1.4)

DS = 10

abs_mag_assigned = cat["Mz"]

abs_mag_max, abs_mag_min = -26, -22.25

mask_model = cat[f"LRG_opt"]==True

ax.hexbin(np.log10(cat["zstarve_smoothed"][mask_model]), cat[f"r-z_{cs_tag}"][mask_model], gridsize=500, bins="log")#, color="r", s=5, alpha=0.05)
ax.scatter(np.log10(cat["zstarve_smoothed"][mask_model])[::DS], cat["r-z"][mask_model][::DS], color="white", s=1, alpha=0.05)

plt.tight_layout()
plt.show()


In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(16,8))

# ax = axes[0]
# ax.semilogx()
# ax.set_xlim(0.4,10)
# ax.set_ylim(-0.05,1.05)

# ax1 = ax.twiny()
# if band=="Mz":
#     ax1.set_xlim(0,1.6)
# elif band=="MW1":
#     ax1.set_xlim(0.8,3.5)

# plot_colors = get_colors(10)
# c = 0

# # colors_by_mag_bin = {}
# color_cdfs   = {}
# zstarve_cdfs = {}

# for i in range(len(abs_mag_bins)-1):
#     abs_mag_min, abs_mag_max = abs_mag_bins[i:i+2]
#     abs_mag_bin_label = f"{abs_mag_min:.2f}_{abs_mag_max:.2f}".replace('.','p').replace('-','n')
    
#     abs_mag_mask_model = (abs_mag_assigned < abs_mag_min) & (abs_mag_assigned >= abs_mag_max)
    
#     this_bin_zstarve = zstarve_smoothed[abs_mag_mask_model]
#     # print(abs_mag_min, len(this_bin_zstarve))
#     H,_ = np.histogram(this_bin_zstarve, bins=zstarve_bins)
#     this_bin_zstarve_cdf = np.cumsum(H)/np.cumsum(H)[-1]

#     this_bin_abs_mag_mask_data = (abs_mag_data < abs_mag_min) & (abs_mag_data >= abs_mag_max)
    
#     this_bin_color = data_colors[this_bin_abs_mag_mask_data]
#     H,_ = np.histogram(this_bin_color, bins=color_bins)
#     this_bin_color_cdf = np.cumsum(H)/np.cumsum(H)[-1]

#     #-- interpolated halo CDF position as function of z_starve
#     #-- input: z_starve
#     #-- infer: halo CDF position
#     halo_cdf_of_zstarve = interpolate.interp1d(zstarve_cens, this_bin_zstarve_cdf, fill_value="extrapolate")

#     #-- interpolated r-? color as function of r-? CDF position
#     #-- input: color CDF position
#     #-- infer: color
#     color_of_color_cdf = interpolate.interp1d(this_bin_color_cdf, color_cens, fill_value="extrapolate")

#     #-- inferred halo CDF position for given z_starve
#     halo_cdf_inferred = halo_cdf_of_zstarve( this_bin_zstarve )

#     #-- assigned (inferred) color for given color CDF position
#     halo_color_inferred = color_of_color_cdf( halo_cdf_inferred )
    
#     cat[f"r-{band[1:]}"][abs_mag_mask_model] = halo_color_inferred

#     if (i+1)%10==0:
#         axes[0].plot(zstarve_cens, this_bin_zstarve_cdf, label=f"({abs_mag_min}, {abs_mag_max})", color=plot_colors[c])
#         ax1.plot(color_cens, this_bin_color_cdf, label="", color=plot_colors[c], ls=":", lw=2)
#         axes[1].hist(halo_color_inferred, bins=color_bins, alpha=0.1, color=plot_colors[c])
#         axes[1].hist(halo_color_inferred, bins=color_bins, histtype="step", color=plot_colors[c])
#         c += 1

# #     color_cdfs[abs_mag_bin_label]   = this_bin_color_cdf
# #     zstarve_cdfs[abs_mag_bin_label] = this_bin_zstarve_cdf
# #     if len(halo_color_inferred) > 0:
# #         colors_by_mag_bin[abs_mag_bin_label] = cat[abs_mag_mask_model]

# axes[0].legend()
# plt.tight_layout()
# plt.show()


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8,8))

# ax.hist(cat[f"r-{band[1:]}"][cat[band] <= abs_mag_lim], bins=color_bins, density=True, color="gray", alpha=0.5, label="model")

# ax.hist( colors_data[abs_mag_mask_data], bins=color_bins, density=True, histtype="step", color="k", lw=2, label="target")

# ax.legend()

# ax.set_xlabel(f"$r-{band[1:]}$")
# ax.set_xlim(0,4)

# plt.tight_layout()
# plt.show()


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8,8))

# DS = 50
# ax.scatter( t[band][::DS], data_colors[::DS], s=2, alpha=0.1 )
# ax.scatter( cat[band][::DS], cat[f"r-{band[1:]}"][::DS], s=2, alpha=0.1 )

# ax.set_xlim(abs_mag_lim+1.0, -26)
# ax.set_ylim(0, 3.5)

# plt.tight_layout()
# plt.show()


In [ ]:
# %%time

# #-- reflag LRGs with improved model color assignment

# #-- add columns for LRG flagging
# for c in (f"LRG_IR", f"LRG_opt"):
#     if c in cat.colnames:
#         cat.remove_column(c)
#     cat.add_column(Column(np.zeros(len(cat), dtype=bool), name=c))

# #-- get model magnitudes and colors for LRG flagging
# model_mags    = cat[band]
# model_mag_min = np.min(model_mags)
# model_mag_max = np.max(model_mags)

# model_colors    = cat[f"r-{band[1:]}"]
# model_color_min = np.min(model_colors)
# model_color_max = np.max(model_colors)


# ### OPTICAL LRG SELECTION ###

# lrgfrac_opt_fname = f"{DATADIR}/stats/lrgfrac_optical-select_{band[1:]}-band_{cat_tag}.npy"
# print(f"Loading {lrgfrac_opt_fname}...")

# mag_bins_opt, color_bins_opt, H_lrg_opt, H_full_opt = np.load(lrgfrac_opt_fname, allow_pickle=True)

# if model_mag_min < np.min(mag_bins_opt):
#     mag_bins_opt[0] = np.floor(model_mag_min)
# if model_mag_max > np.max(mag_bins_opt):
#     mag_bins_opt[-1] = model_mag_max

# if model_color_min < np.min(color_bins_opt):
#     color_bins_opt[0] = np.floor(model_color_min)
# if model_color_max > np.max(color_bins_opt):
#     color_bins_opt[-1] = model_color_max

# lrg_frac_opt = H_lrg_opt/H_full_opt

# N_bins = (len(color_bins_opt)-1)*(len(mag_bins_opt)-1)

# k = 1
# cat["LRG_opt"] = False

# for i in range(len(color_bins_opt)-1):
#     color_mask = (model_colors > color_bins_opt[i]) & (model_colors <= color_bins_opt[i+1])
#     for j in range(len(mag_bins_opt)-1):
#         mag_mask = (model_mags > mag_bins_opt[j]) & (model_mags <= mag_bins_opt[j+1])
#         bin_mask = mag_mask & color_mask
#         N_mock   = len(cat[bin_mask])
#         if N_mock > 0:
#             indices = np.where(bin_mask)[0]            
#             N_gal_opt = H_full_opt[i,j]
#             N_lrg_opt = H_lrg_opt[i,j]

#             targ_lrg_opt = 0
#             if (N_lrg_opt > 0) & (N_gal_opt > 0):
#                 targ_lrg_opt = int(N_mock*lrg_frac_opt[i,j])

#             if (targ_lrg_opt > 0):
#                 if (targ_lrg_opt < N_mock):
#                     selected = indices[np.random.choice(len(indices), targ_lrg_opt, replace=False)]
#                     cat["LRG_opt"][selected] = True
#                 else:
#                     cat["LRG_opt"][indices] = True

#             if (targ_lrg_opt > 0) and (k%100==0):
#                 print(f"{k}/{N_bins}\t{color_bins_opt[i]:.2f}\t{mag_bins_opt[j]:.2f}\t{N_mock}\t{targ_lrg_opt}")
#         k += 1


# ### IR LRG SELECTION ###

# lrgfrac_IR_fname = f"{DATADIR}/stats/lrgfrac_IR-select_{band[1:]}-band_{cat_tag}.npy"
# print(f"Loading {lrgfrac_IR_fname}...")

# mag_bins_IR, color_bins_IR, H_lrg_IR, H_full_IR = np.load(lrgfrac_IR_fname, allow_pickle=True)

# if model_mag_min < np.min(mag_bins_IR):
#     mag_bins_IR[0] = np.floor(model_mag_min)
# if model_mag_max > np.max(mag_bins_IR):
#     mag_bins_IR[-1] = model_mag_max

# if model_color_min < np.min(color_bins_IR):
#     color_bins_IR[0] = np.floor(model_color_min)
# if model_color_max > np.max(color_bins_IR):
#     color_bins_IR[-1] = model_color_max

# lrg_frac_IR = H_lrg_IR/H_full_IR

# N_bins = (len(color_bins_IR)-1)*(len(mag_bins_IR)-1)

# k = 1
# cat["LRG_IR"] = False

# for i in range(len(color_bins_IR)-1):
#     color_mask = (model_colors > color_bins_IR[i]) & (model_colors <= color_bins_IR[i+1])
#     for j in range(len(mag_bins_IR)-1):
#         mag_mask = (model_mags > mag_bins_IR[j]) & (model_mags <= mag_bins_IR[j+1])
#         bin_mask = mag_mask & color_mask
#         N_mock   = len(cat[bin_mask])
#         if N_mock > 0:
#             indices = np.where(bin_mask)[0]            
#             N_gal_IR = H_full_IR[i,j]
#             N_lrg_IR = H_lrg_IR[i,j]

#             targ_lrg_IR = 0
#             if (N_lrg_IR > 0) & (N_gal_IR > 0):
#                 targ_lrg_IR = int(N_mock*lrg_frac_IR[i,j])

#             if (targ_lrg_IR > 0):
#                 if (targ_lrg_IR < N_mock):
#                     selected = indices[np.random.choice(len(indices), targ_lrg_IR, replace=False)]
#                     cat["LRG_IR"][selected] = True
#                 else:
#                     cat["LRG_IR"][indices] = True

#             if (targ_lrg_IR > 0) and (k%100==0):
#                 print(f"{k}/{N_bins}\t{color_bins_IR[i]:.2f}\t{mag_bins_IR[j]:.2f}\t{N_mock}\t{targ_lrg_IR}")
#         k += 1


<a class="anchor" id="color-mag-dist-fig"></a>
## Color and magnitude distribution comparison figure

[BACK TO TOP](#outline)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5.5))

c_opt, c_IR = get_colors(2)

lrg_opt = cat["LRG_opt"]==True
lrg_IR  = cat["LRG_IR"]==True

zphot_label = f"$({zmin}$" + r"$<z_{\rm phot}<$" + f"${zmax})$"

#-- data vs model COLORS
ax = axes[0]
H_full,_ = np.histogram(colors_data[abs_mag_mask_data], bins=color_bins)
ax.plot(color_cens, H_full/np.sum(H_full), color="k", label="data " + zphot_label)
H_opt,_ = np.histogram(colors_data[abs_mag_mask_data & (t["LRG_opt"]==True) ], bins=color_bins)
ax.plot(color_cens, H_opt/np.sum(H_full), color=c_opt, label="")
H_IR,_ = np.histogram(colors_data[abs_mag_mask_data & (t["LRG_IR"]==True) ], bins=color_bins)
ax.plot(color_cens, H_IR/np.sum(H_full), color="orangered", label="")

H_full,_ = np.histogram(cat[f"r-{band[1:]}"], bins=color_bins)
ax.fill_between(color_cens, H_full/np.sum(H_full), color="gray", alpha=0.3, lw=2, label="parent sample mock \n(magnitude limited)")
H_opt,_ = np.histogram(cat[f"r-{band[1:]}"][lrg_opt], bins=color_bins)
ax.fill_between(color_cens, H_opt/np.sum(H_full), color=c_opt, alpha=0.4, lw=3, label="mock optical LRG")
H_IR,_ = np.histogram(cat[f"r-{band[1:]}"][lrg_IR], bins=color_bins)
ax.fill_between(color_cens, H_IR/np.sum(H_full), color=c_IR, lw=3, alpha=0.5, label="mock IR LRG")

ax.set_xlabel(f"$r-{band[1:]}$", fontsize=24)
ax.set_ylabel("relative distribution", fontsize=20)
xlim = (0,1.65) if band=="Mz" else (0.5,3.6)
ax.set_xlim(xlim)
xticks = np.arange(0,1.61,0.2) if band=="Mz" else np.arange(0.5,3.51,0.5)
ax.set_xticks(xticks)
ax.set_ylim(0, 1.05*np.max(H_full/np.sum(H_full)))

zsim_label = r"$z_{\rm sim}=$" + f"${zsim:.3f}$"
ha = "right" if ((zmin==0.4) & (band != "Mz")) else "left"
ax.text(*get_corners(ax)[f"upper_{ha}"], zsim_label, va="top", ha=f"{ha}", fontsize=24)


#-- data vs model MAGNITUDES
ax = axes[1]
bins = np.arange(-26,-21,0.1) if band=="Mz" else np.arange(-26,-21,0.1)
mag_cens = 0.5*(bins[1:] + bins[:-1])

s = len(t)/len(t[t[band] <= abs_mag_lim])
H_full,_ = np.histogram(t[band], bins=bins)
mask = (mag_cens <= abs_mag_lim)
ax.plot(mag_cens, s*H_full/np.sum(H_full), color="k", ls=":", label="")
ax.plot(mag_cens[mask], (s*H_full/np.sum(H_full))[mask], color="k", label=f"data {zphot_label}")

H_full,_ = np.histogram(t[band][abs_mag_mask_data], bins=bins)
H_opt,_ = np.histogram(t[band][abs_mag_mask_data & (t["LRG_opt"]==True) ], bins=bins)
ax.plot(mag_cens, H_opt/np.sum(H_full), color=c_opt, label="")
H_IR,_ = np.histogram(t[band][abs_mag_mask_data & (t["LRG_IR"]==True) ], bins=bins)
ax.plot(mag_cens, H_IR/np.sum(H_full), color="orangered", label="")

H_full,_ = np.histogram(cat[band], bins=bins)
ax.fill_between(mag_cens[mask], (H_full/np.sum(H_full))[mask], color="gray", alpha=0.3, lw=2, label="parent sample mock \n(magnitude limited)")
H_opt,_ = np.histogram(cat[band][lrg_opt], bins=bins)
ax.fill_between(mag_cens, H_opt/np.sum(H_full), color=c_opt, alpha=0.4, lw=3, label="mock optical LRG")
H_IR,_ = np.histogram(cat[band][lrg_IR], bins=bins)
ax.fill_between(mag_cens, H_IR/np.sum(H_full), color=c_IR, lw=3, alpha=0.5, label="mock IR LRG")

xlabel = "$M_z$" if band=="Mz" else "$M_{W1}$"
ax.set_xlabel(xlabel, fontsize=24)
xlim = (-21.2,-24.6) if band=="Mz" else (-22.2,-26)
ax.set_xlim(xlim)
ax.set_ylim(0, 1.05*np.max(H_full/np.sum(H_full)))

ax.legend(loc=1, fontsize=17, labelspacing=0.3, handletextpad=0.3, markerfirst=False)

for ax in axes:
    ax.tick_params(axis="both", which="major", labelsize=18)
    ax.set_yticks([])

plt.tight_layout()
plt.subplots_adjust(wspace=0.025)

fname = f"{BASEDIR}/figures/color-mag-dist-compare_{cat_tag[:-6]}_{abs_mag_lim_tag}.png"
print(fname)

#plt.savefig(fname, bbox_inches="tight", pad_inches=0.1, dpi=300)

plt.show()


<a class="anchor" id="lum-func-fig"></a>
## Luminosity function comparison figure

[BACK TO TOP](#outline)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5.5))

c_opt, c_IR = get_colors(2)

lrg_opt = cat["LRG_opt"]==True
lrg_IR  = cat["LRG_IR"]==True

lrg_opt_data = t["LRG_opt"]==True
lrg_IR_data  = t["LRG_IR"]==True

xlim = (-21.5,-24.5) if band=="Mz" else (-22.15,-25.5)
ax.set_xlim(xlim)
ax.set_ylim(-0.01,1.05)

xlabel = "$M_z$" if band=="Mz" else "$M_{W1}$"
ax.set_xlabel(xlabel, fontsize=20)
if band=="MW1":
    ax.set_xticks(np.arange(-25.5,-22,0.5))
ax.set_ylabel("cumulative distribution", fontsize=20)

abs_mag_bins = np.arange(-26,-21,0.1)
mag_cens = 0.5*(abs_mag_bins[1:] + abs_mag_bins[:-1])

H,_ = np.histogram(t[band][t[band] < abs_mag_lim], bins=abs_mag_bins)
cdf = 1-np.cumsum(H)/np.max(np.cumsum(H))
ax.plot(mag_cens, cdf, color="k", label="parent sample", ls="--")
ax.fill_between(mag_cens, cdf, color="gray", alpha=0.1)

H,_ = np.histogram(t[band][lrg_opt_data], bins=abs_mag_bins)
cdf = 1-np.cumsum(H)/np.max(np.cumsum(H))
ax.plot(mag_cens, cdf, color=c_opt, label=f"optical LRG", ls="dashdot", lw=1.5)
ax.fill_between(mag_cens, cdf, color=c_opt, alpha=0.1)

H,_ = np.histogram(t[band][lrg_IR_data], bins=abs_mag_bins)
cdf = 1-np.cumsum(H)/np.max(np.cumsum(H))
ax.plot(mag_cens, cdf, color=c_IR, label=f"IR LRG", lw=2)
ax.fill_between(mag_cens, cdf, color=c_IR, alpha=0.15)

ax.plot(abs_mag_bins_clust[0]*np.ones(2), ax.get_ylim(), color="gray", ls=":", lw=2)
# ax.plot(abs_mag_bins_clust[3]*np.ones(2), ax.get_ylim(), color="gray", ls=":", lw=2)
for i in abs_mag_bins_clust[1:4]:
    ax.plot(i*np.ones(2), (-1,0.92), color="gray", ls=":", lw=2)
    ax.plot(i*np.ones(2), (1.0,1.1), color="gray", ls=":", lw=2)

#loc=(0.025,0.625)
ax.legend(loc=4, fontsize=18, labelspacing=0.3, handletextpad=0.3, frameon=True, markerfirst=False)

zphot_label = f"${zmin}$" + r"$<z_{\rm phot}<$" + f"${zmax}$"
# xmin, xmax = ax.get_xlim()
# xt = xmax - 0.05*(xmax-xmin)
ax.text(*get_corners(ax)["upper_left"], zphot_label, fontsize=20, ha="left", va="top")

ax.tick_params(axis="both", which="major", labelsize=16)

plt.tight_layout()

fname = f"{BASEDIR}/figures/lum-func-compare_{cat_tag[:-6]}_{abs_mag_lim_tag}.png"
print(fname)

plt.savefig(fname, bbox_inches="tight", pad_inches=0.1, dpi=300)

plt.show()


In [ ]:
#-- save mock with improved color assignment and LRGs flagged

galcat_fname = f"{BASEDIR}/mocks/{sim_tag}/{sham_tag}/{d}/{zsnap_tag}_{abs_mag_lim_tag}_galcat-hist-LRG.npy"

print(galcat_fname)

np.save(galcat_fname, cat)


<a class="anchor" id="color_scatter"></a>
## Add scatter to colors

[BACK TO TOP](#outline)


In [ ]:
colors_by_mag_bin = {}

for i in range(len(abs_mag_bins)-1):
    abs_mag_min, abs_mag_max = abs_mag_bins[i:i+2]
    abs_mag_bin_label = f"{abs_mag_min:.2f}_{abs_mag_max:.2f}".replace('.','p').replace('-','n')
    
    abs_mag_mask_model = (cat[band] < abs_mag_min) & (cat[band] >= abs_mag_max)
    
    colors_by_mag_bin[abs_mag_bin_label] = cat[abs_mag_mask_model]


In [ ]:
%%time

#-- add scatter to assigned colors

# color_sig_array = (0.05, 0.10, 0.20, 0.40)
color_sig_array = (0.6, 0.8, 1.0)

color_cname = f"r-{band[1:]}"

for sigma_color in color_sig_array:
    sigc = str(round(sigma_color,2)).replace('.','p')
    print(sigc)
    
    for k in colors_by_mag_bin.keys():
        this_mag_bin_cat    = colors_by_mag_bin[k]
        this_mag_bin_colors = this_mag_bin_cat[color_cname]

        this_mag_bin_cat.sort(color_cname)

        colors_ranked = this_mag_bin_cat[color_cname].data.copy()

        colors_ranked_scattered = colors_ranked + np.random.normal(0, sigma_color, len(colors_ranked))

        if "crs" in this_mag_bin_cat.colnames:
            this_mag_bin_cat.remove_column("crs")
        this_mag_bin_cat.add_column(Column(data=colors_ranked_scattered, name="crs"))

        this_mag_bin_cat.sort("crs")

        if f"{color_cname}_cs{sigc}" in this_mag_bin_cat.colnames:
            this_mag_bin_cat.remove_column(f"{color_cname}_cs{sigc}")

        this_mag_bin_cat.add_column(Column(data=colors_ranked, name=f"{color_cname}_cs{sigc}"))

        if "crs" in this_mag_bin_cat.colnames:
            this_mag_bin_cat.remove_column("crs")

test_cat = vstack(list(colors_by_mag_bin.values()))

len(test_cat)==len(cat)



In [ ]:
# def cont(ax, x, y, color="black", alpha=0.1):
#     xlim = ax.get_xlim()
#     ylim = ax.get_ylim()
#     if xlim[0] > xlim[1]:
#         xlim = xlim[::-1]
#     if ylim[0] > ylim[1]:
#         ylim = ylim[::-1]
#     H, X, Y = np.histogram2d( x, y, bins=(40,40), range=(xlim,ylim) )
#     Hflat = np.concatenate(H)
#     XX = 0.5*(X[:-1]+X[1:])
#     YY = 0.5*(Y[:-1]+Y[1:])
#     pctiles = np.arange(60,101,20)
#     levels  = np.percentile(Hflat[Hflat > 0],pctiles)
#     ax.contour(XX, YY, H.T, linewidths=1, levels=levels, colors=color)
#     ax.contourf(XX, YY, H.T, levels=levels, alpha=alpha, colors=color)

# mask = test_cat["LRG_IR"]==True

# fig, ax = plt.subplots(1, 1, figsize=(6,6))

# ax.set_xlim(0,4)
# ax.set_ylim(0,4)

# for sigc in color_sig_array[::-1]:
#     sigc = str(np.round(sigc,2)).replace('.','p')
#     color_cname = f"r-{band[1:]}"
#     # ax.scatter(test_cat[color_cname], test_cat[f"{color_cname}_cs{sigc}"], s=3, alpha=0.2, color="gray")
#     cont(ax, test_cat[color_cname], test_cat[f"{color_cname}_cs{sigc}"], color="white", alpha=1)
#     cont(ax, test_cat[color_cname], test_cat[f"{color_cname}_cs{sigc}"], color="k")

# cont(ax, test_cat[color_cname][mask], test_cat[f"{color_cname}_cs{sigc}"][mask], color="purple")

# ax.plot(ax.get_xlim(), ax.get_ylim(), color="black", lw=1, ls="--")
# plt.show()


In [ ]:
# bins = np.linspace(0,4,121)

# mask = test_cat["LRG_IR"]==True

# # plt.hist(cat[color_cname], bins=bins, color="gray", alpha=0.5)
# plt.hist(test_cat[color_cname][mask], bins=bins, color="purple", alpha=0.5)

# # plt.hist(cat[f"{color_cname}_cs1p0"], bins=bins, histtype="step", color="black", lw=2)
# plt.hist(test_cat[f"{color_cname}_cs0p05"][mask], bins=bins, histtype="step", color="purple", lw=2)

# #plt.hist(test_cat["crs"], bins=np.linspace(0,3,31), histtype="step", color="red", lw=2)

# plt.show()



<a class="anchor" id="lrg"></a>
# LRG flagging

[BACK TO TOP](#outline)


In [ ]:
%%time

for sigma_color in color_sig_array:
    sigc = str(round(sigma_color,2)).replace('.','p')
    print(sigc)

    #-- add columns for LRG flagging
    for c in (f"LRG_IR_cs{sigc}", f"LRG_opt_cs{sigc}"):
        if c in test_cat.colnames:
            test_cat.remove_column(c)
        test_cat.add_column(Column(np.zeros(len(test_cat), dtype=bool), name=c))

    #-- get model magnitudes and colors for LRG flagging
    model_mags    = test_cat[band]
    model_mag_min = np.min(model_mags)
    model_mag_max = np.max(model_mags)

    model_colors    = test_cat[f"r-{band[1:]}_cs{sigc}"]
    model_color_min = np.min(model_colors)
    model_color_max = np.max(model_colors)


    ### OPTICAL LRG SELECTION ###

    lrgfrac_opt_fname = f"{DATADIR}/stats/lrgfrac_optical-select_{band[1:]}-band_{cat_tag}.npy"
    print(f"Loading {lrgfrac_opt_fname}...")

    mag_bins_opt, color_bins_opt, H_lrg_opt, H_full_opt = np.load(lrgfrac_opt_fname, allow_pickle=True)

    if model_mag_min < np.min(mag_bins_opt):
        mag_bins_opt[0] = np.floor(model_mag_min)
    if model_mag_max > np.max(mag_bins_opt):
        mag_bins_opt[-1] = model_mag_max

    if model_color_min < np.min(color_bins_opt):
        color_bins_opt[0] = np.floor(model_color_min)
    if model_color_max > np.max(color_bins_opt):
        color_bins_opt[-1] = model_color_max

    lrg_frac_opt = H_lrg_opt/H_full_opt

    N_bins = (len(color_bins_opt)-1)*(len(mag_bins_opt)-1)

    k = 1
    test_cat[f"LRG_opt_cs{sigc}"] = False

    for i in range(len(color_bins_opt)-1):
        color_mask = (model_colors > color_bins_opt[i]) & (model_colors <= color_bins_opt[i+1])
        for j in range(len(mag_bins_opt)-1):
            mag_mask = (model_mags > mag_bins_opt[j]) & (model_mags <= mag_bins_opt[j+1])
            bin_mask = mag_mask & color_mask
            N_mock   = len(test_cat[bin_mask])
            if N_mock > 0:
                indices = np.where(bin_mask)[0]            
                N_gal_opt = H_full_opt[i,j]
                N_lrg_opt = H_lrg_opt[i,j]

                targ_lrg_opt = 0
                if (N_lrg_opt > 0) & (N_gal_opt > 0):
                    targ_lrg_opt = int(N_mock*lrg_frac_opt[i,j])

                if (targ_lrg_opt > 0):
                    if (targ_lrg_opt < N_mock):
                        selected = indices[np.random.choice(len(indices), targ_lrg_opt, replace=False)]
                        test_cat[f"LRG_opt_cs{sigc}"][selected] = True
                    else:
                        test_cat[f"LRG_opt_cs{sigc}"][indices] = True

                if (targ_lrg_opt > 0) and (k%100==0):
                    print(f"{k}/{N_bins}\t{color_bins_opt[i]:.2f}\t{mag_bins_opt[j]:.2f}\t{N_mock}\t{targ_lrg_opt}")
            k += 1


    ### IR LRG SELECTION ###

    lrgfrac_IR_fname = f"{DATADIR}/stats/lrgfrac_IR-select_{band[1:]}-band_{cat_tag}.npy"
    print(f"Loading {lrgfrac_IR_fname}...")

    mag_bins_IR, color_bins_IR, H_lrg_IR, H_full_IR = np.load(lrgfrac_IR_fname, allow_pickle=True)

    if model_mag_min < np.min(mag_bins_IR):
        mag_bins_IR[0] = np.floor(model_mag_min)
    if model_mag_max > np.max(mag_bins_IR):
        mag_bins_IR[-1] = model_mag_max

    if model_color_min < np.min(color_bins_IR):
        color_bins_IR[0] = np.floor(model_color_min)
    if model_color_max > np.max(color_bins_IR):
        color_bins_IR[-1] = model_color_max

    lrg_frac_IR = H_lrg_IR/H_full_IR

    N_bins = (len(color_bins_IR)-1)*(len(mag_bins_IR)-1)

    k = 1
    test_cat[f"LRG_IR_cs{sigc}"] = False

    for i in range(len(color_bins_IR)-1):
        color_mask = (model_colors > color_bins_IR[i]) & (model_colors <= color_bins_IR[i+1])
        for j in range(len(mag_bins_IR)-1):
            mag_mask = (model_mags > mag_bins_IR[j]) & (model_mags <= mag_bins_IR[j+1])
            bin_mask = mag_mask & color_mask
            N_mock   = len(test_cat[bin_mask])
            if N_mock > 0:
                indices = np.where(bin_mask)[0]            
                N_gal_IR = H_full_IR[i,j]
                N_lrg_IR = H_lrg_IR[i,j]

                targ_lrg_IR = 0
                if (N_lrg_IR > 0) & (N_gal_IR > 0):
                    targ_lrg_IR = int(N_mock*lrg_frac_IR[i,j])

                if (targ_lrg_IR > 0):
                    if (targ_lrg_IR < N_mock):
                        selected = indices[np.random.choice(len(indices), targ_lrg_IR, replace=False)]
                        test_cat[f"LRG_IR_cs{sigc}"][selected] = True
                    else:
                        test_cat[f"LRG_IR_cs{sigc}"][indices] = True

                if (targ_lrg_IR > 0) and (k%100==0):
                    print(f"{k}/{N_bins}\t{color_bins_IR[i]:.2f}\t{mag_bins_IR[j]:.2f}\t{N_mock}\t{targ_lrg_IR}")
            k += 1

save_as = f"{BASEDIR}/mocks/{sim_tag}/{sham_tag}/{d}/color_scatter/{zsnap_tag}_{abs_mag_lim_tag}_galcat-LRG.npy"
print(f"Saving {save_as}...")

np.save(save_as, test_cat)


In [ ]:
test_cat


In [ ]:
print(len(test_cat[test_cat["LRG_IR"]==True]))
print(len(test_cat[test_cat["LRG_opt"]==True]))

for c in test_cat.colnames:
    if ("LRG_" in c) & ("_cs" in c):
        print(f"{c}\t{len(test_cat[test_cat[c]==True])}")
        
        